# Calulate NECC transport in INALT20 and observation along 23$^{\circ}$W

For both, model and observations we calculate the central position $ Y_{CM} $ and along-pathway intensity $ INT $ of zonal currents using the algorithm of Hsin (2012). 

\begin{equation}
Y_{CM}(x,t) = \frac{\int_{Z_l}^{Z_u} \int_{Y_{S}}^{Y_{N}} y\ u(x,y,z,t)\ dy\ dz}{\int_{Z_l}^{Z_u} \int_{Y_{S}}^{Y_{N}} u(x,y,z,t)\ dy\ dz}
\label{equ_Y_CM}
\end{equation}

\begin{equation}
INT(x,t) = \int_{Z_l}^{Z_u} \int_{Y_{CM}-W}^{Y_{CM}+W} u(x,y,z,t)\ dy\ dz 
\label{equ_INT}
\end{equation}


where $y$ is latitude, $x$ is longitude, $u$ is zonal velocity, $z$ is depth, $t$ is time, $Z_u$ ($Z_l$) is upper (lower) boundary of the flow, $Y_N$ ($Y_S$) is northern (southern) limit of the flow, and $W$ is the half mean width of the flow.

![](../figures/INALT20_obs_23w_comparison/1_INALT20_obs_23w_1999_2012.png)

For transport calculation of the NECC we chose the following boundary conditions:

$Z_u = 0\,$m, $Z_l = 24.5\,$kg$\,$m$^{-3}$, $Y_N = 10^{\circ}$N, $Y_S = 4^{\circ}$S

Based on monthly mean CORE climatology (INATL20_NECC_NECC_boundaries) we choose $W = 3.5^{\circ}$ with max extent from 3$^{\circ}$ to 10$^{\circ}$N.

The depth of the NECC core $Z_{CM}$ will be estimated similar to $Y_{CM}$:

\begin{equation}
Z_{CM}(x,t) = \frac{\int_{Z_l}^{Z_u} \int_{Y_{S}}^{Y_{N}} z\ u(x,y,z,t)\ dy\ dz}{\int_{Z_l}^{Z_u} \int_{Y_{S}}^{Y_{N}} u(x,y,z,t)\ dy\ dz}
\label{equ_Z_CM}
\end{equation}

Think about variable depth boundaries. Or boundaries define by density. Because is the model we calculate the NECC transport across the basin, not only at 23W.

## Tech preample

In [1]:
%matplotlib inline
import os
import dask
import cmocean
import datetime
import nc_time_axis
import cftime
import time
import seawater as sw
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from pathlib import Path
from scipy.io import loadmat
from datetime import datetime

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Dask jobqueue and client
To controle the resources used for parallel computations on computing nodes.

In [3]:
import dask_jobqueue
cluster = dask_jobqueue.SLURMCluster(
    # Dask worker size
    cores=4, memory='24GB',
    processes=1, # Dask workers per job
    # SLURM job script things
    queue='cluster', walltime='01:30:00',
    # Dask worker network and temporary storage
    interface='ib0', local_directory='$TMPDIR',
    log_directory='./slurm_logs'
)

In [4]:
from dask.distributed import Client

In [5]:
client = Client(cluster)
client

Client Scheduler: tcp://172.18.4.13:42271 Dashboard: http://172.18.4.13:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
cluster.adapt(
    minimum=1, maximum=20,
)

## Output parameters

In [7]:
script_name = 'INALT20_NECC_calc_transport'
out_dir_data = Path('../data/'+script_name+'/')
Path(out_dir_data).mkdir(parents=True, exist_ok=True)
out_data_1 = 'INALT20_NECC_transport'

out_dir_fig = Path('../figures/'+script_name+'/')
Path(out_dir_fig).mkdir(parents=True, exist_ok=True)
out_fig_1 = 'INALT20_'
fig_format = '.png'

savefig = 1; #set one if figures should be saved

## Input parameters
For INALT20 

In [8]:
run_calc_JRA = 1 #set 1 if JRA should be recalculated
run_calc_CORE = 0 #set 1 if CORE should be recalculated
global_data_path = Path("/sfs/fs1/work-geomar1/smomw044/")
JRA_path = "INALT20.L46-KFS10X"
exp_id = "INALT20.L46-KFS10?"

CORE_path = "shared/INALT20.L46-KFS044-S"
CORE_exp_id = "INALT20.L46-KFS044" # INALT COREv2

temp_res = "_5d_" # 5d:5-daily; 1m:monthly; 1y:yearly; 1d:daily(stored currently on TAPE) 
nest_prefix = "1_" # "1_" for high resolution; leave empty for base model

# chunk sizes
chu_x = 100 # None means take the full dataset
chu_y = 100
chu_z = None
chu_t = 1

# variables wanted
vars_want = ['vosaline','votemper','vozocrtx']

Calculation of EUC - Boundary conditions

In [9]:
lonlim = [-42, -17]
latlim = [3, 10]
zlim = [0, 100]

## integers
xclim1 = 549
xclim2 = 1050
yclim1 = 1690
yclim2 = 1831

p_ref = 0 # dbar; reference pressure for potential density calculation
W = 3.5 #degN, half width of flow
sigma_lim = [0,24.5] #kg/m^3, vertical boundaries of flow for INT [Z_u, Z_l]

Find relevant data files for INALT20

## Functions

### Potential density

In [10]:
def coords_to_str(latlim,lonlim):
    lats_str=f'{abs(latlim[0])}s' if latlim[0] < 0 else f'{latlim[0]}n'
    latn_str=f'{abs(latlim[1])}s' if latlim[1] < 0 else f'{latlim[1]}n'
    lonw_str=f'{abs(lonlim[0])}w' if lonlim[0] < 0 else f'{lonlim[0]}e'
    lone_str=f'{abs(lonlim[1])}w' if lonlim[1] < 0 else f'{lonlim[1]}e'
    return lats_str,latn_str,lonw_str,lone_str

In [11]:
def calc_pdens_sw(p_ref,ds):
    
    """ Calculates potential density using the EOS-80 seawater library
    Parameters
    ----------
    pref : int
    reference pressure for potential density calculation
    ds : xr.DataSet containing NEMO3 model output
      ds.deptht  : xr.DataArray (depth in m, positive downwards)
      ds.gphit  : xr.DataArray (latitude grid in degN)
      ds.vosaline  : xr.DataArray (practical salinity(eos 80))
      ds.votemper  : xr.DataArray (potential temp (eos 80))
   
    Returns
    -------
    xr.DataArray
    Data array of potential density calculated using the EOS-80 seawater library [kg/m^3], same dimension as input data arrays
    """
    
    # calculate pressure
    p = xr.apply_ufunc(
    sw.eos80.pres,
    -abs(ds.deptht),ds.gphit,
    dask='parallelized', output_dtypes=[float, ]
    )
    
    # calculate in-situ temperature from potential temperature
    t = xr.apply_ufunc(sw.eos80.temp,
                       ds.vosaline,ds.votemper,p,p_ref,
                       dask = 'parallelized',output_dtypes=[float,])
    
    # compute potential density
    sig = xr.apply_ufunc(sw.eos80.pden,
                         ds.vosaline,t,p,p_ref,
                         dask = 'parallelized',output_dtypes=[float,])
    sig -= 1000
    
    sig['tmask'] = ds.tmask
    sig = sig.where(sig.tmask==1)
    sig.name='sigma_%sm' %p_ref
    sig.attrs['units']='kg/m^3'
    sig.attrs['long_name']='Potential density'
    sig.attrs['reference_pressure']= '%s dbar' %p_ref
    
    return sig

### Eastward transport

In [12]:
def eastw_transport(ucur,dy,dz,z_dim,y_dim):
    ucur = ucur.where(ucur>=0)
    trs_e = (dy*(ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)*1e-6
    
    trs_e.attrs['units']='Sv'
    trs_e.attrs['long_name']='Eastward transport'
    return trs_e

### Y$_{CM}$ (central position of a current)

In [13]:
def YCM(ucur,sigma,lat,dy,dz,sigma_lim,z_dim,y_dim):
    ucur = ucur.where((ucur>=0)&
                      (sigma_lim[0]<=sigma)&
                      (sigma<=sigma_lim[1]))
    y_cm = (dy*(lat*ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)/(
            dy*(ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)
    
    y_cm.attrs['units']='degN'
    y_cm.attrs['name']='Y_CM'
    y_cm.attrs['long_name'] = 'Central latitude'
    return y_cm

### Z$_{CM}$ (core depth of a current)

In [14]:
def ZCM(ucur,sigma,z,dy,dz,sigma_lim,z_dim,y_dim):
    ucur = ucur.where((ucur>=0)&
                      (sigma_lim[0]<=sigma)&
                      (sigma<=sigma_lim[1]))
    z_cm = (dy*(z*ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)/(
            dy*(ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)
    
    z_cm.attrs['units']='m'
    z_cm.attrs['name']='Z_CM'
    z_cm.attrs['long_name'] = 'Central depth'
    return z_cm

### INT (eastward transport using cor following algorithm of Hsin 2012)

In [15]:
def eastw_INT(ucur,sigma,Y_CM,depth,lat,dy,dz,sigma_lim,W,z_dim,y_dim):
    ucur = ucur.where(((ucur>0)&
                      (sigma_lim[0]<=sigma)&
                      (sigma<=sigma_lim[1])&
                      (Y_CM-W < lat)&
                      (Y_CM+W > lat)))
    INT_e = (dy*(ucur*dz).sum(dim=z_dim)).sum(dim=y_dim,skipna=True)*1e-6
    
    INT_e.attrs['units']='Sv'
    INT_e.attrs['name']='INT'
    INT_e.attrs['long_name']='Eastward along-pathway intensity'
    INT_e.attrs['history']='Eastward along-pathway intensity'
    return INT_e

In [16]:
aux_files = list(sorted(
        (global_data_path/ JRA_path).glob(f"{nest_prefix}[m,n]*.nc")
    ))

aux_files

with dask.config.set(scheduler='synchronous'):
    ds_mesh = xr.open_dataset(
            aux_files[0],
            decode_cf=True,
            chunks={"t":chu_t,"z":chu_z, 
                    "y":chu_y,"x":chu_x})

In [17]:
# lonlim = [-42, -17]
# latlim = [-3, 10]
# zlim = [0, 400]

# ## integers
# xclim1 = 549
# xclim2 = 1050
# yclim1 = 1690
# yclim2 = 1831
# ds_mesh.gphit.sel(x=600,y=slice(1690,1831)).squeeze().values
# ds_mesh.glamt.sel(y=1700,x=slice(549,1050)).squeeze().values

## Run calculations

In [18]:
def load_INALT20_calc_transp(JRA_files,ds_mesh,model_str,chu_x,chu_y,chu_z,chu_t,
                             xclim1,xclim2,yclim1,yclim2,sigma_lim,p_ref,W_orig,
                            latlim,lonlim):
    with dask.config.set(scheduler='synchronous'):
        ds_JRA_T = xr.open_dataset(
            JRA_files[0],
            chunks={"time_counter":chu_t,
                     "deptht":chu_z, 
                     "y":chu_y, 
                     "x":chu_x},
            decode_cf=True,
            )
    ds_JRA_T = ds_JRA_T.get(['vosaline','votemper'])

    with dask.config.set(scheduler='synchronous'):
        ds_JRA_U = xr.open_dataset(
        JRA_files[1],
        decode_cf=True,
        chunks={"time_counter":chu_t,
                 "depthu":chu_z, 
                 "y":chu_y, 
                 "x":chu_x},)


    ## select region JRA
    ds_JRA_T = ds_JRA_T.assign_coords(gphit=(['y','x'],ds_mesh.gphit.squeeze()))
    ds_JRA_T = ds_JRA_T.assign_coords(glamt=(['y','x'],ds_mesh.glamt.squeeze()))
    ds_JRA_T = ds_JRA_T.drop(['nav_lat','nav_lon'])

    ds_JRA_U = ds_JRA_U.assign_coords(gphiu=(['y','x'],ds_mesh.gphiu.squeeze()))
    ds_JRA_U = ds_JRA_U.assign_coords(glamu=(['y','x'],ds_mesh.glamu.squeeze()))
    ds_JRA_U = ds_JRA_U.drop(['nav_lat','nav_lon'])

    ds_JRA_U['e1u'] = (('y', 'x'), ds_mesh.e1u.squeeze())
    ds_JRA_U['e2u'] = (('y', 'x'), ds_mesh.e2u.squeeze())
    ds_JRA_U['e3u'] = (('depthu','y', 'x'), ds_mesh.e3u_0.squeeze())

    ds_JRA_T.coords['tmask'] = ds_mesh.rename_dims({'z':'deptht'}).tmask.squeeze()
    ds_JRA_U.coords['umask'] = ds_mesh.rename_dims({'z':'depthu'}).umask.squeeze()

    ds_JRA_T = ds_JRA_T.sel(deptht=slice(0,400),
                           x=slice(xclim1,xclim2),
                           y=slice(yclim1,yclim2))

    ds_JRA_U = ds_JRA_U.sel(depthu=slice(0,400),
                           x=slice(xclim1,xclim2),
                           y=slice(yclim1,yclim2))

    # calculate density        
    pdens_JRA = calc_pdens_sw(p_ref,ds_JRA_T)

    ds_JRA_U = ds_JRA_U.isel(x=slice(0,-1)).swap_dims({'depthu':'depth'}
                    ).rename({'depthu':'depth'})

    with xr.set_options(keep_attrs=True):
        # average on U-Grid
        pdens_JRA =(pdens_JRA.isel(x=slice(None,-1))+pdens_JRA.isel(x=slice(1,None)))/2
        # merge to one dataset
        box_JRA = xr.merge([ds_JRA_U.vozocrtx.where(ds_JRA_U.umask==1),
                    ds_JRA_U.e1u,
                    ds_JRA_U.e2u,
                    ds_JRA_U.e3u,
                    pdens_JRA.swap_dims({'deptht':'depth'}
                    ).rename({'deptht':'depth'})])
    
    # set coordinates and attributes
    lon = box_JRA.glamu.isel(y=5)
    box_JRA = box_JRA.assign_coords(lon=('x',lon))
    lat = box_JRA.gphit.isel(x=400)
    box_JRA = box_JRA.assign_coords(lat=('y',lat))
    box_JRA = box_JRA.swap_dims({'x':'lon','y':'lat'}
                     ).drop({'gphiu','gphit','glamu','time_centered'})
    box_JRA.lat.attrs['units']='degN'
    box_JRA.lat.attrs['long_name']='Latitude'
    box_JRA.lon.attrs['units']='degE'
    box_JRA.lon.attrs['long_name']='Longitude'
    box_JRA.depth.attrs['units']='m'
    box_JRA.depth.attrs['long_name']='Depth'

    box_JRA.load()
    lat_s,lat_n,lon_w,lon_e=coords_to_str(latlim,lonlim)

    box_JRA.to_netcdf((out_dir_data / f"{JRA_files[1].name[:-9]}lat{lat_s}{lat_n}_lon{lon_w}{lon_e}.nc"),
    engine='netcdf4',
    encoding={'time_counter':{'units':'days since 1900-01-01 00:00:00'}})
    time.sleep(10)

    ## actual calulations
    ds = box_JRA.sel(lat=slice(4,10))
    Y_CM = YCM(ds.vozocrtx,ds.sigma_0m,ds.lat,ds.e2u,ds.e3u,sigma_lim,
               'depth','lat')
    Y_CM.attrs['lat_lim'] = '4degN to 10degN'
    Y_CM.attrs['depth_lim'] = 'Z_u to Z_l (pot dens)'
    Y_CM.attrs['Z_u'] = sigma_lim[0]
    Y_CM.attrs['Z_u_name'] = 'Upper boundary of flow'
    Y_CM.attrs['Z_u_unit'] = 'kg/m^3'
    Y_CM.attrs['Z_l'] = sigma_lim[1]
    Y_CM.attrs['Z_l_name'] = 'Lower boundary of flow'
    Y_CM.attrs['Z_l_unit'] = 'kg/m^3'

    Z_CM = ZCM(ds.vozocrtx,ds.sigma_0m,ds.depth,ds.e2u,ds.e3u,sigma_lim,
               'depth','lat')
    Z_CM.attrs['lat_lim'] = '4degN to 10degN'
    Z_CM.attrs['depth_lim'] = 'Z_u to Z_l (pot dens)'
    Z_CM.attrs['Z_u'] = sigma_lim[0]
    Z_CM.attrs['Z_u_name'] = 'Upper boundary of flow'
    Z_CM.attrs['Z_u_unit'] = 'kg/m^3'
    Z_CM.attrs['Z_l'] = sigma_lim[1]
    Z_CM.attrs['Z_l_name'] = 'Lower boundary of flow'
    Z_CM.attrs['Z_l_unit'] = 'kg/m^3'

    ds = box_JRA.sel(lat=slice(3,10),depth=slice(0,65))
    T_fixed = eastw_transport(ds.vozocrtx,ds.e2u,ds.e3u,
                              'depth','lat')
    T_fixed.attrs['lat_lim'] = '3 to 10degN'
    T_fixed.attrs['depth_lim'] = '0m to 65m'

    INT = eastw_INT(box_JRA.vozocrtx,box_JRA.sigma_0m,Y_CM,ds.depth,box_JRA.lat,
              box_JRA.e2u,box_JRA.e3u,sigma_lim,W,'depth','lat')
    INT.attrs['lat_lim'] = 'Y_CM-W (min 3degN) to Y_CM+W (max 10degN)'
    INT.attrs['W_name'] = 'Half mean width of flow'
    INT.attrs['W_unit'] = 'degN'
    INT.attrs['W'] = W
    INT.attrs['depth_lim'] = 'Z_u to Z_l (pot dens)'
    INT.attrs['Z_u'] = sigma_lim[0]
    INT.attrs['Z_u_name'] = 'Upper boundary of flow'
    INT.attrs['Z_u_unit'] = 'kg/m^3'
    INT.attrs['Z_l'] = sigma_lim[1]
    INT.attrs['Z_l_name'] = 'Lower boundary of flow'
    INT.attrs['Z_l_unit'] = 'kg/m^3'

    INT_25 = eastw_INT(box_JRA.vozocrtx,box_JRA.sigma_0m,Y_CM,ds.depth,box_JRA.lat,
              box_JRA.e2u,box_JRA.e3u,[sigma_lim[0], 25.5],W,'depth','lat')
    INT_25.attrs['lat_lim'] = 'Y_CM-W (min 3degN) to Y_CM+W (max 10degN)'
    INT_25.attrs['W_name'] = 'Half mean width of flow'
    INT_25.attrs['W_unit'] = 'degN'
    INT_25.attrs['W'] = W_orig
    INT_25.attrs['depth_lim'] = 'Z_u to Z_l (pot dens)'
    INT_25.attrs['Z_u'] = sigma_lim[0]
    INT_25.attrs['Z_u_name'] = 'Upper boundary of flow'
    INT_25.attrs['Z_u_unit'] = 'kg/m^3'
    INT_25.attrs['Z_l'] = 25.5
    INT_25.attrs['Z_l_name'] = 'Lower boundary of flow'
    INT_25.attrs['Z_l_unit'] = 'kg/m^3'


    with xr.set_options(keep_attrs=True):
        NECC_JRA = xr.merge([
            Y_CM.rename('Y_CM'),Z_CM.rename('Z_CM'),
            INT.rename('INT'),INT_25.rename('INT_25'),T_fixed.rename('T_fixed'),
        ])


    NECC_JRA.attrs['title'] = f'NECC transports in INALT20 {model_str}'
    NECC_JRA.attrs['timeStamp'] = '%s' % datetime.now()
    NECC_JRA.attrs['history'] = 'Original model output modified using INALT20_NECC_calc_transport.ipynb'

    NECC_JRA.to_netcdf((out_dir_data / f"{JRA_files[1].name[:-9]}{out_data_1}{'.nc'}"),
    engine='netcdf4',
    encoding={'time_counter':{'units':'days since 1900-01-01 00:00:00'}})
    return NECC_JRA,box_JRA

In [19]:
%%time
for restr_years in range(2007,2019):
    print (restr_years)
    # JRA
    JRA_files = list(sorted(
        (global_data_path / JRA_path ).glob(
            f"{nest_prefix}{exp_id}{temp_res}{restr_years}????_{restr_years}????_grid_[T,U].nc")
    ))

    NECC_JRA,box_JRA = load_INALT20_calc_transp(JRA_files,ds_mesh,'JRA',chu_x,chu_y,chu_z,chu_t,
                        xclim1,xclim2,yclim1,yclim2,sigma_lim,p_ref,W,
                        latlim,lonlim)
    time.sleep(10) 

    # CORE
    if restr_years<2010:
        CORE_files = list(sorted(
            (global_data_path / CORE_path ).glob(
                f"{nest_prefix}{CORE_exp_id}{temp_res}{restr_years}????_{restr_years}????_grid_[T,U].nc"
            )))
        NECC_CORE,box_CORE = load_INALT20_calc_transp(CORE_files,ds_mesh,'CORE',chu_x,chu_y,chu_z,chu_t,
                            xclim1,xclim2,yclim1,yclim2,sigma_lim,p_ref,W,
                            latlim,lonlim)
        time.sleep(10)

cluster.close()

2007


distributed.utils - ERROR - Timed out during handshake while connecting to tcp://172.18.4.185:42507 after 10 s
Traceback (most recent call last):
  File "/gxfs_home/geomar/smomw294/miniconda3/envs/xorca_env/lib/python3.9/site-packages/distributed/comm/tcp.py", line 187, in read
    n_frames = await stream.read_bytes(8)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/gxfs_home/geomar/smomw294/miniconda3/envs/xorca_env/lib/python3.9/asyncio/tasks.py", line 489, in wait_for
    fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/gxfs_home/geomar/smomw294/miniconda3/envs/xorca_env/lib/python3.9/site-packages/distributed/comm/core.py", line 319, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/gxfs_home/geomar/smomw294/miniconda3/envs/xorca_env/lib/

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
CPU times: user 13min 50s, sys: 3min 32s, total: 17min 22s
Wall time: 1h 10min 3s


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
